In [1]:
import cupy as np
import pyDR as DR
from pyDR.Project import Project
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [2]:
test = "test_004"

In [3]:
tf=4000
molsys=DR.MolSys('/mnt/c/Users/thier/Desktop/1ubq_frame.pdb',
                 '/home/mlehner/1ubq/'+test+'/run_001/traj_4us_PBC_fit.xtc',
                 tf=tf)
select=DR.MolSelect(molsys)

In [4]:
ile_resn = [3, 13, 23, 30, 36, 44, 61]

In [5]:
def get_frame_analysis(resnum):
    frames=list()
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':resnum,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':resnum,'n_bonds':1,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':resnum,'n_bonds':2,'sigma':0})

    "Frames with post-process smoothing"
    frames.append({'Type':'hops_3site','Nuc':'ivl', 'resids':resnum,'sigma':5})
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':resnum,'sigma':5})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':resnum,'n_bonds':1,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':resnum,'n_bonds':1,'sigma':50})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':resnum,'n_bonds':2,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':resnum,'n_bonds':2,'sigma':50})
    select.select_bond(Nuc='ivl', resids=resnum)
    fr_obj=DR.Frames.FrameObj(select)
    fr_obj.tensor_frame(sel1=1,sel2=2)
    for f in frames:fr_obj.new_frame(**f)
    fr_obj.load_frames(n=-1)
    fr_obj.post_process()
    include=np.zeros([3,9],dtype=bool)
    include[0][:3]=True  
    include[1][[4,6,8]]=True
    include[2][3:]=True
    t=np.arange(tf>>1)*.005
    data=fr_obj.frames2data(mode='full')
    proj=Project('/home/mlehner/pyDR/pyDR/test_project.nosync',create=True)
    for d in data:proj.append_data(d)
    proj[0].detect.r_no_opt(10)
    proj.fit(bounds=False)
    proj['no_opt'][0].detect.r_auto(5)
    proj['no_opt'].fit()
    return include, fr_obj, t, proj
    

In [6]:
data_ile = []
for resnum in ile_resn:
    include, fr_obj, t, proj = get_frame_analysis(resnum)
    data_ile.append([resnum, include, fr_obj, t, proj])
    print(f"Residue {resnum} done")

TypeError: Argument 'array' has incorrect type (expected cupy.core.core.ndarray, got numpy.ndarray)

In [ ]:
for resnum, include, fr_obj, t, proj in data_ile:
    titles=[['Methyl rot.',r'$\chi_2$ rot.',r'$\chi_1$ rot.',r'C$\alpha$-C$\beta$ motion','Total'],
            ['Methyl rot.',r'$\chi_2$ rot.',r'$\chi_1$ rot.',r'C$\alpha$-C$\beta$ motion','Total'],
            ['Methyl lib.','Methyl hop.',r'$\chi_2$ lib.',r'$\chi_2$ hop.',
            r'$\chi_1$ lib.',r'$\chi_1$ hop.',r'C$\alpha$-C$\beta$ motion','Total']]
    fig=plt.figure('Methyl Dynamics')
    fig.clear()
    fig.set_size_inches([10.27,  9.03])
    ax=[]
    ax.append([fig.add_subplot(5,3,k+1) for k in range(0,15,3)])
    ax.append([fig.add_subplot(5,3,k+1) for k in range(1,15,3)])
    ax.append([fig.add_subplot(5,6,k+1) for k in [4,5,10,11,16,17,22]])
    ax[-1].append(fig.add_subplot(5,3,15))

    for a0,t0 in zip(ax,titles):
        for a, title in zip(a0,t0):
            a.set_title(title)

    for inc,ax0 in zip(include,ax):
        out=fr_obj.frames2ct(include=inc,mode='full')
        
        for ct,a in zip(out['ct_finF'],ax0):
            try:
                a.cla()
                a.plot(t,ct.mean(0)[:tf>>1])
                a.set_ylim([0,1.05])
                S2=ct.mean(0)[tf>>2:tf>>1].mean()
                b=np.argwhere(ct.mean(0)-S2<0)[0,0]
                tc0=np.max([.001,((ct.mean(0)[:b]-S2)/(1-S2)).sum()*.005])
                #tc0=t[np.argmin(np.abs((ct.mean(0)[:b]-S2)/(1-S2)-np.exp(-1)))]
                fun=lambda x:(((x[0]+(1-x[0])*np.exp(-t[:b]/x[1]))-ct.mean(0)[:b])**2).sum()
                S2,tc=least_squares(fun,[S2,tc0]).x
                a.plot(t,S2+(1-S2)*np.exp(-t/tc),color='grey',linestyle=':')
                a.set_xlim([0,np.min([10*tc,fr_obj.t[tf>>1]])])
            except:
                pass
        ax0[-1].semilogx(out['t'][:tf>>1],out['ct'].mean(0)[:tf>>1])
        ax0[-1].semilogx(out['t'][:tf>>1],out['ct_prod'].mean(0)[:tf>>1])
        ax0[-1].set_ylim([0,.5])
        
    fig.set_size_inches([180/25.4,220/25.4])

    plt.savefig("/home/mlehner/pyDR/out_figures/ile/"+test+"/frame_ile_"+str(resnum)+".pdf")

    plt.show()

In [ ]:
fig = plt.figure()
ax = fig.gca()
for resnum, include, fr_obj, t, proj in data_ile:
    out = fr_obj.frames2ct(include=include[-1], mode='full')
    print(resnum, out["ct_prod"].shape)
    print(out["ct_finF"].shape)
    #plt.semilogx(out['t'][:tf>>1],out['ct'].mean(0)[:tf>>1])
    plt.semilogx(out['t'][:tf>>1],out['ct_prod'].mean(0)[:tf>>1])
    ax.set_ylim([0,.5])
plt.legend(ile_resn)
plt.savefig("/home/mlehner/pyDR/out_figures/ile/"+test+"/overview.pdf")